In [77]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Set current working directory
print(os.getcwd())
#current_directory = os.getcwd()
current_directory = "/share/data/analyses/ahmet/rms_specs_deep"
os.chdir(current_directory)
print(os.getcwd())

/share/data/analyses/ahmet/rms_specs_deep
/share/data/analyses/ahmet/rms_specs_deep


In [80]:
import numpy as np
import pandas as pd
import cv2
import albumentations as album
from tqdm import tqdm

# Load the image metadata
df = pd.read_csv('images_metadata_channels.csv', delimiter=';')

# Shuffle the data
df = df.sample(frac=1, random_state=1)

# Specify the path for the shuffled DataFrame CSV file
shuffled_csv_path = 'shuffled_images_metadata_channels.csv'

# Save the shuffled DataFrame to a CSV file
df.to_csv(shuffled_csv_path, sep=';', index=False)

# Load DMSO statistics with a multi-level header for direct access
dmso_stats_df = pd.read_csv('rh30_dmso_channel_stats_final.csv', delimiter=';', header=[0,1], index_col=0)

# Function for DMSO normalization
def dmso_normalization(im, dmso_mean, dmso_mad):
    im_norm = (im.astype('float') - dmso_mean) / dmso_mad
    return im_norm

# Transformation function for resizing images
image_size = 256
easy_transforms = album.Compose([album.Resize(image_size, image_size)])

# A function for transfering image to numpy array 
def create_all_images(idx):
    row = df.iloc[idx]
    im = []
    for i in range(1,6):
        local_im = cv2.imread(row['w' + str(i)], -1)
        dmso_mean = dmso_stats_df[row.Metadata_Barcode]['w' + str(i)]['Median']
        dmso_mad = dmso_stats_df[row.Metadata_Barcode]['w' + str(i)]['MAD']
        local_im = dmso_normalization(local_im, dmso_mean, dmso_mad)

        im.append(local_im)
    im = np.array(im).transpose(1, 2, 0).astype("float")
    im = np.array(easy_transforms(image = im)['image'])

    return im

import logging

# Setup logging
logging.basicConfig(filename='logfile.log', level=logging.INFO, format='%(asctime)s - %(message)s')

# Write all the images into a big numpy array  
all_images = np.zeros((df.shape[0], 256, 256, 5), dtype = np.float32)
for f in tqdm(range(df.shape[0]), desc='Processing Images'):  # Wrap range with tqdm for a progress bar
    all_images[f] = create_all_images(f)
    logging.info(f"Processed image {f+1}/{df.shape[0]}")

# Save the big numpy array 
with open('all_images.npy', 'wb') as f:
    np.save(f, all_images)
    


Processing Images:   0%|          | 301/108591 [04:28<26:50:00,  1.12it/s]


KeyboardInterrupt: 